# Mounting Google Drive

In [1]:
##### before running it, make sure you don't have lots of big files in your google drive
##### otherwise it's going to take too long to finish running it before giving the TIMEOUT error
##### also save the train_controls, train_patients, val_controls, val_patients to your drive and
##### create a "train" folder with train_controls, train_patients in it, and 
##### a "val" folder with val_controls, val_patients in it.
##### change the train_dir and val_dir in the next cell to the dir of your train and val folder
##### e.g. my train and val folder are saved in the "deep learning" dir

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Loading Train, Val, and Test Data Using Keras ImageDataGenerator

In [2]:
#Kavi's and Chelsea's Dir
train_dir = "/gdrive/My Drive/kaggle_dataset/train/"
val_dir = "/gdrive/My Drive/kaggle_dataset/val/"
test_dir = "/gdrive/My Drive/kaggle_dataset/test/"

# Daniel's dir:
# train_dir = "/gdrive/My Drive/deep learning/Project/trainProbMaps/"
# val_dir = "/gdrive/My Drive/deep learning/Project/valProbMaps/"
# test_dir = "/gdrive/My Drive/deep learning/Project/testProbMaps/"

img_width = 224
img_height = 224
batch_size = 1
channels = 3
epochs = 50
nb_train_samples = 8000
nb_valid_samples = 32
nb_test_samples = 968

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)             
valid_datagen = ImageDataGenerator(rescale=1./255)    
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)   

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True) #weight toward one class or another

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Using TensorFlow backend.


Found 8000 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


# Building the Model Architecture (Pre-trained Keras VGG16 Extracting Features from OCT Dataset) & Training the Model

In [4]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential

from PIL import Image

Image.MAX_IMAGE_PIXELS = None

from keras import layers

from keras.callbacks import TensorBoard

#from keras.layers import Input, Dense
from keras import layers
from keras.applications import resnet50
from keras import optimizers
from keras.applications import VGG16

#pretrained VGG16 on imagenet, starting to replicate Hassan's work
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_height, img_width, channels))

#conv_base.summary()

#Extracting features from OCT data using pretrained VGG
def extract_features(dataset_type, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count))
    i = 0
    if dataset_type == "train":
        for inputs_batch, labels_batch in train_generator:
            features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break   
    elif dataset_type == "valid":
        for inputs_batch, labels_batch in valid_generator:
            features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break
    else:
        for inputs_batch, labels_batch in test_generator:
            features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = features_batch
            labels[i * batch_size : (i + 1) * batch_size] = labels_batch
            i += 1
            if i * batch_size >= sample_count:
                break
    return features, labels

train_features, train_labels = extract_features("train", nb_train_samples)
valid_features, valid_labels = extract_features("valid", nb_valid_samples)
test_features, test_labels = extract_features("test", nb_test_samples)


print(train_features.shape, train_labels.shape)
print(valid_features.shape, valid_labels.shape)
print(test_features.shape, test_labels.shape)

(8000, 7, 7, 512) (8000,)
(32, 7, 7, 512) (32,)
(968, 7, 7, 512) (968,)


# For hidden

In [0]:
import keras

#img_width = 475
#img_height = 388

img_width = 224
img_height = 224

from PIL import Image

Image.MAX_IMAGE_PIXELS = None

#EGD data
egd_dir = "/gdrive/My Drive/kaggle_dataset/test/"

egd_datagen = ImageDataGenerator(rescale=1./255)

egd_generator = egd_datagen.flow_from_directory(
    egd_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_height, img_width, channels))

conv_base.summary()
# inp = keras.layers.Input(shape=(img_height, img_width, channels), name='image_input')

# vgg_model = VGG16(weights='imagenet',
#                   include_top=False)
# vgg_model.trainable = False

# x = keras.layers.Flatten(name='flatten')(vgg_model)
# x = keras.layers.Dense(4096, activation='relu', name='fc1')(x)
# x = keras.layers.Dense(4096, activation='relu', name='fc2')(x)
# x = keras.layers.Dense(1000, activation='softmax', name='predictions')(x)
# new_model = keras.models.Model(inputs=inp, outputs=x)
# new_model.compile(optimizer='adam', loss='categorical_crossentropy', 
#                   metrics=['accuracy'])

# new_model.fit_generator(
#     egd_generator,
#     steps_per_epoch=102 / batch_size,
#     epochs=epochs,
#      validation_data=validation_generator,
#      validation_steps=nb_validation_samples / batch_size,
#     shuffle=True) # // batch_size)



features = np.zeros(shape=(968, 7, 7, 512))
labels = np.zeros(shape=(968))
i = 0
for inputs_batch, labels_batch in egd_generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= 968:
        break
        
print(features.shape, labels.shape)

from google.colab import files
features = np.reshape(features, (968, 7*7*512))
print(features.shape)
np.savetxt('Labels.txt', labels)
files.download('Labels.txt')
np.savetxt('VGG_RF_Features.txt', features)
files.download('VGG_RF_Features.txt')

Found 968 images belonging to 4 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None,

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 49150, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
# divide to 10-folds

# produce 10-fold cross validation and train the model 10 times to get the average val_accuracy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics

kf = KFold(n_splits=10)
kf.get_n_splits(features)

fold_num = 0
count = 0

score_list = []
auc_list = []

for train_index, val_index in kf.split(features):
    clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
    clf.fit(np.reshape(features[train_index], (len(train_index), 7*7*512)), labels[train_index])
    valid_prediction = clf.predict(np.reshape(features[val_index], (len(val_index), 7*7*512)))
    valid_score = clf.score(np.reshape(features[val_index], (len(val_index), 7*7*512)), labels[val_index])
    fpr, tpr, thresholds = metrics.roc_curve(labels[val_index], valid_prediction, pos_label=1)
    valid_auc = metrics.auc(fpr, tpr)
    score_list.append(valid_score)
    auc_list.append(valid_auc)
    
print("mean score is: {}".format(np.mean(score_list)))
print("mean AUC is: {}".format(np.mean(auc_list)))


mean score is: 0.8884664948453608
mean AUC is: 0.3588617281058046


# Classifier Layer: Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
clf.fit(np.reshape(train_features, (8000, 7*7*512)), train_labels)

valid_prediction = clf.predict(np.reshape(valid_features, (32, 7*7*512)))
# print("validation accuracy:", sum([prediction[i] == valid_labels[i] for i in range(len(valid_labels))])/len(valid_labels))
#print(prediction)
#print(valid_labels)

test_prediction = clf.predict(np.reshape(test_features, (968, 7*7*512)))
# print("test accuracy:", sum([prediction[i] == test_labels[i] for i in range(len(test_labels))])/len(test_labels))
# print(test_prediction)
# print(test_labels)


In [7]:
valid_score = clf.score(np.reshape(valid_features, (nb_valid_samples, 7*7*512)), valid_labels)
test_score = clf.score(np.reshape(test_features, (nb_test_samples, 7*7*512)), test_labels)

print("valid accuracy:", valid_score)
print("test accuracy:", test_score)

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(valid_labels, valid_prediction, pos_label=1)
print("valid AUC:", metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(test_labels, test_prediction, pos_label=1)
print("test AUC:", metrics.auc(fpr, tpr))

valid accuracy: 0.90625
test accuracy: 0.893595041322314
valid AUC: 0.43229166666666663
test AUC: 0.4221962297657264


In [0]:
print(test_generator.filenames)

In [0]:
control_list, patient_list = test_generator.filenames[:100], test_generator.filenames[100:]

print(test_generator.filenames[:100])
print(test_generator.filenames[100:])

control_idx_list = [9, 61, 65, 70]
patient_idx_list = [32, 43, 46, 48, 52, 86]

from google.colab import files

patient_name_list = [patient_list[i] for i in patient_idx_list]
control_name_list = [control_list[i] for i in control_idx_list]

for i in range(len(patient_name_list)):
    files.download(test_dir+patient_name_list[i])

for i in range(len(control_name_list)):
    files.download(test_dir+control_name_list[i])

# Testing

In [0]:
#print(train_generator.filenames)
#print(validation_generator.filenames)

nb_test_samples = 102

result = model.evaluate_generator(test_generator, steps=nb_test_samples / batch_size)
print(result)

predictions = model.predict_generator(test_generator, steps=nb_test_samples / batch_size)
print(predictions)
print(len(predictions))
img_path = "/gdrive/My Drive/validation/val_patients/patient13.png"

from keras.preprocessing import image                             
import numpy as np

img = image.load_img(img_path, target_size=(img_width, img_height))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.                                               


print(img_tensor.shape)
#validation_generator.classes

NameError: ignored

# Visualizing

In [0]:
import matplotlib.pyplot as plt

#plt.imshow(img_tensor[0])
#plt.show()

from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]               
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_tensor)
first_layer_activation = activations[0]
print(first_layer_activation.shape)

import matplotlib.pyplot as plt
plt.figure()
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

# Visualizing attention map

In [0]:
# !pip install keras-vis
!pip install git+https://github.com/raghakot/keras-vis.git

from keras.applications import ResNet50
from vis.utils import utils
from keras import activations

# Hide warnings on Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')

In [0]:
from vis.utils import utils
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

# img1 = utils.load_img('/gdrive/My Drive/deep_learning/val/val_controls/_ERGO-0449_OS_2014_triton_wf_report.png', 
#                       target_size=(img_width / 5, img_height / 5))
# img2 = utils.load_img('/gdrive/My Drive/deep_learning/val/val_controls/_ERGO-0450_OD_2014_triton_wf_report.png', 
#                       target_size=(img_width / 5, img_height / 5))

img1 = utils.load_img('/gdrive/My Drive/deep_learning/new_dataset/test/controls/View2098.png', 
                      target_size=(img_width, img_height))
img2 = utils.load_img('/gdrive/My Drive/deep_learning/new_dataset/train/patients/194.png', 
                      target_size=(img_width, img_height))

f, ax = plt.subplots(1, 2)
ax[0].imshow(img1)
ax[1].imshow(img2)

In [0]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations
from vis.visualization import visualize_cam


# import numpy as np
import matplotlib.cm as cm
# from vis.visualization import visualize_cam


penultimate_layer = utils.find_layer_idx(model, 'conv2d_15')
layer_idx = utils.find_layer_idx(model, 'dense_9')

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 4)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img1, img2]):    
        # 20 is the imagenet index corresponding to `ouzel`
        grads = visualize_cam(model, layer_idx, filter_indices=0, 
                              seed_input=img, penultimate_layer_idx=penultimate_layer,
                              backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
        ax[i].imshow(jet_heatmap)
        ax[i + 2].imshow(img)
#         ax[i].imshow(jet_heatmap)
#         print(jet_heatmap.shape)
#         print(img.shape)
#         ax[i].imshow(overlay(jet_heatmap, img))
